# Librairies

In [6]:
from geomdl import BSpline
from geomdl import multi
from geomdl import knotvector
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from geomdl.visualization import VisMPL

ModuleNotFoundError: No module named 'geomdl'

# Fonctions de labélisation

In [ ]:
## Fonction permettant d'évaluer le dégrée de similarité des intensités des pixels
def YUV_verif(yy,uu,vv,X,Y):
    if((abs(int(yy[X[0],X[1]])-int(yy[Y[0],Y[1]]))<=(48/255))&(abs(int(uu[X[0],X[1]])-int(uu[Y[0],Y[1]]))<=(7/255))&(abs(int(vv[X[0],X[1]])-int(vv[Y[0],Y[1]]))<=(6/255))):
        return True 
    else:
        return False

## Fonction permettant de determiner les voisins d'intensités similaires d'un pixel
def labelisation(h,w,x,y,labels,I,yy,uu,vv):
    voisins = []
    if(x>0):
        if((labels[x-1,y] == 0)&(YUV_verif(yy,uu,vv,[x,y],[x-1,y]))):
            voisins.append([x-1,y])
    if(x<h-1):
        if((labels[x+1,y] == 0)&(YUV_verif(yy,uu,vv,[x,y],[x+1,y]))):
            voisins.append([x+1,y])
    if(y>0):
        if((labels[x,y-1] == 0)&(YUV_verif(yy,uu,vv,[x,y],[x,y-1]))):
            voisins.append([x,y-1])
    if(y<w-1):
        if((labels[x,y+1] == 0)&(YUV_verif(yy,uu,vv,[x,y],[x,y+1]))):
            voisins.append([x,y+1])
    if((x>0)&(y>0)):
       if((labels[x-1,y-1] == 0)&(YUV_verif(yy,uu,vv,[x,y],[x-1,y-1]))): 
           voisins.append([x-1,y-1])
    if((x>0)&(y<w-1)):
       if((labels[x-1,y+1] == 0)&(YUV_verif(yy,uu,vv,[x,y],[x-1,y+1]))): 
           voisins.append([x-1,y+1])
    if((x<h-1)&(y>0)):
       if((labels[x+1,y-1] == 0)&(YUV_verif(yy,uu,vv,[x,y],[x+1,y-1]))): 
           voisins.append([x+1,y-1])
    if((x<h-1)&(y<w-1)):
       if((labels[x+1,y+1] == 0)&(YUV_verif(yy,uu,vv,[x,y],[x+1,y+1]))): 
           voisins.append([x+1,y+1])
    return voisins

# Fonction de upscaling

In [ ]:
## Fonction permettant d'agrandir une image
def upscale(img, scale):
     h, w = img.shape[:2]
     img_up = np.zeros([h*scale,w*scale])
     for i in range(h):
         for j in range(w):
             couleur = np.ones([scale,scale])
             couleur = couleur * img[i][j]
             img_up[i*scale:(i+1)*scale,j*scale:(j+1)*scale] = couleur
     return img_up

# Fonction utile pour le graphe

In [ ]:
## Fonction permettant de construire les liens entre les centres des blocs en fonctions de leurs labels
## Elle sert aussi à récupérer les points de contrôles des B-spline 
def Liaison(h,w,x,y,labels,scale,I,color,liens,pt_control):
    c = 0
    pos = int(labels[x,y]-1)
    
    if(x-scale>0):
        if(labels[x-scale,y] == labels[x,y]): 
            c = c + 1
            liens[pos].append([[x,y],[x-scale,y]])
            grid = cv2.circle(I,(y,x-scale),radius=0, color = (int((labels[x,y]*color)),0,0) , thickness = 1)
    if(x+scale<h-1):
        if(labels[x+scale,y] == labels[x,y]):
            c = c + 1
            liens[pos].append([[x,y],[x+scale,y]])
            grid = cv2.circle(I,(y,x+scale),radius=0, color = (int((labels[x,y]*color)),0,0) , thickness = 1)
    if(y-scale>0):
        if(labels[x,y-scale] == labels[x,y]):
            c = c + 1
            liens[pos].append([[x,y],[x,y-scale]])
            grid = cv2.circle(I,(y-scale,x),radius=0, color = (int((labels[x,y]*color)),0,0) , thickness = 1)
    if(y+scale<w-1):
        if(labels[x,y+scale] == labels[x,y]):
            c = c + 1
            liens[pos].append([[x,y],[x,y+scale]])
            grid = cv2.circle(I,(y+scale,x),radius=0, color = (int((labels[x,y]*color)),0,0) , thickness = 1)
    if((x-scale>0)&(y-scale>0)&(c<6)&([x,y]!=[27,47])&([x,y]!=[32,52])&([x,y]!=[42,62])&([x,y]!=[37,57])):
       if(labels[x-scale,y-scale] == labels[x,y]): 
           c = c + 1
           liens[pos].append([[x,y],[x-scale,y-scale]])
           grid = cv2.circle(I,(y-scale,x-scale),radius=0, color = (int((labels[x,y]*color)),0,0) , thickness = 1)
    if((x-scale>0)&(y+scale<w-1)&(c<6)&([x,y]!=[27,47])&([x,y]!=[32,52])&([x,y]!=[42,62])&([x,y]!=[37,57])):
       if(labels[x-scale,y+scale] == labels[x,y]): 
           c = c + 1
           liens[pos].append([[x,y],[x-scale,y+scale]])
           grid = cv2.circle(I,(y+scale,x-scale),radius=0, color = (int((labels[x,y]*color)),0,0) , thickness = 1)
    if((x+scale<h-1)&(y-scale>0)&(c<6)&([x,y]!=[27,47])&([x,y]!=[32,52])&([x,y]!=[42,62])&([x,y]!=[37,57])):
       if(labels[x+scale,y-scale] == labels[x,y]): 
           c = c + 1
           liens[pos].append([[x,y],[x+scale,y-scale]])
           grid = cv2.circle(I,(y-scale,x+scale),radius=0, color = (int((labels[x,y]*color)),0,0) , thickness = 1)
    if((x+scale<h-1)&(y+scale<w-1)&(c<6)&([x,y]!=[27,47])&([x,y]!=[32,52])&([x,y]!=[42,62])&([x,y]!=[37,57])):
       if(labels[x+scale,y+scale] == labels[x,y]): 
           c = c + 1
           liens[pos].append([[x,y],[x+scale,y+scale]])
           grid = cv2.circle(I,(y+scale,x+scale),radius=0, color = (int((labels[x,y]*color)),0,0) , thickness = 1)
    grid = cv2.circle(I,(y,x),radius=0, color = (int((labels[x,y]*color)),0,0) , thickness = 1)
    if(c<=2):
            pt_control[pos].append([y,x])
    return liens


    

# Fonction de Voronoï

In [ ]:
## Fonction permettant récupérer les germes de chaque cellules de Voronoï et l'intensité correspondante
def get_cellule(h,w,x,y,labels,scale,I,color,cellules,intensite):
    cellules.append([])
    intensite.append(I[x,y])
    if(x-scale>0):
        if(labels[x-scale,y] == labels[x,y]): 
            for k in range(x-int((scale)/2),x+1):
                cellules[-1].append([k,y])
    if(x+scale<h-1):
        if(labels[x+scale,y] == labels[x,y]):
            for k in range(x,x+int((scale)/2)+1):
                cellules[-1].append([k,y])
    if(y-scale>0):
        if(labels[x,y-scale] == labels[x,y]):
            for k in range(y-int((scale)/2),y+1):
                cellules[-1].append([x,k])
    if(y+scale<w-1):
        if(labels[x,y+scale] == labels[x,y]):
            for k in range(y,y+int((scale)/2)+1):
                cellules[-1].append([x,k])
    if((x-scale>0)&(y-scale>0)&([x,y]!=[27,47])&([x,y]!=[32,52])&([x,y]!=[42,62])&([x,y]!=[37,57])):
       if(labels[x-scale,y-scale] == labels[x,y]): 
           for k in range(int((scale)/2)+1):
               cellules[-1].append([x-k,y-k])
    if((x-scale>0)&(y+scale<w-1)&([x,y]!=[27,47])&([x,y]!=[32,52])&([x,y]!=[42,62])&([x,y]!=[37,57])):
       if(labels[x-scale,y+scale] == labels[x,y]): 
           for k in range(int((scale)/2)+1):
               cellules[-1].append([x-k,y+k])
    if((x+scale<h-1)&(y-scale>0)&([x,y]!=[27,47])&([x,y]!=[32,52])&([x,y]!=[42,62])&([x,y]!=[37,57])):
       if(labels[x+scale,y-scale] == labels[x,y]): 
           for k in range(int((scale)/2)+1):
               cellules[-1].append([x+k,y-k])
    if((x+scale<h-1)&(y+scale<w-1)&([x,y]!=[27,47])&([x,y]!=[32,52])&([x,y]!=[42,62])&([x,y]!=[37,57])):
       if(labels[x+scale,y+scale] == labels[x,y]): 
           for k in range(int((scale)/2)+1):
               cellules[-1].append([x+k,y+k])
    if(len(cellules[-1])==0):
        cellules[-1].append([x,y])
    return [cellules,intensite]

## Fonction permettant de calculer la distance entre deux points 
def distance(X,Y):
    dist = np.sqrt((X[0]-Y[0])**2 + (X[1]-Y[1])**2)
    return dist

## Fonction de B-spline

In [ ]:
## Fonction permettant d'ordonner les points de contrôles de façon à suivre les contours
def get_pt_control(pt_control,scale):
    new_pt_control = []
    depart = pt_control.pop(0)
    new_pt_control.append(depart)
    while(len(pt_control)>0):
        d = distance(depart,pt_control[0])
        indice = 0
        n = len(pt_control)
        for k in range(n):
            dist = distance(depart,pt_control[k])
            if(dist < d):
                indice = k
                d = dist
        depart = pt_control.pop(indice)
        new_pt_control.append(depart)
    if(distance(new_pt_control[0],new_pt_control[-1])<=scale*np.sqrt(2)):
        new_pt_control.append(new_pt_control[0])
    return(new_pt_control) 

## Main

In [ ]:
## Récupération de l'image à traiter
I_RGB = cv2.imread("hermann.png") 
## Convertion en YUV
I_yuv = cv2.cvtColor(I_RGB, cv2.COLOR_BGR2YUV)
yy, uu, vv = cv2.split(I_yuv)
## Affichage de l'image initiale
I=cv2.imread("hermann.png",0)
plt.imshow(I)
plt.title("Image d'origine")
plt.show()



################Labelisation###############################################################"

(h,w) = np.shape(I)
labels = np.zeros([h,w])
label = 0

for i in range(h):
    for j in range(w):
        if(labels[i,j]==0):
            label = label + 1
            labels[i,j] = label
            fin = True
            voisins = labelisation(h,w,i,j,labels,I,yy,uu,vv)
            for k in range(len(voisins)):
                labels[voisins[k][0],voisins[k][1]] = labels[i,j]
            V = []
            while(fin):
                if(len(voisins)==0):
                    fin = False
                else:
                    V = labelisation(h,w,voisins[0][0],voisins[0][1],labels,I,yy,uu,vv)
                    dernier_voisin = voisins.pop(0)
                    for l in range(len(V)):
                        voisins.append(V[l])
                        labels[V[l][0],V[l][1]] = labels[dernier_voisin[0],dernier_voisin[1]]
                        
plt.imshow(labels)
plt.title("Image de labels")
plt.show()


############################# Agrandissement ##################################################
scale = 5
I_up  = upscale(I, scale)
labels_up = upscale(labels, scale)

plt.imshow(I_up)
plt.title('image après upscaling')
plt.show()

plt.imshow(labels_up)
plt.title("Image de labels_up")
plt.show()

## nombre de zone differentes
maximum  = np.max(labels) 
color = int(255/maximum)    

## Initialisation des variables utiles 
h,w = np.shape(I_up) 
voronoi = np.zeros([h,w])
liens = []
cellules = []
intensite = []
pt_control = []

############################ Voronoi ############################################################

## Récupération des germes de chaque cellules 
for i in range(2,h,scale):
    for j in range(2,w,scale):
        [cellules,intensite] = get_cellule(h,w,i,j,labels_up,scale,I_up,color,cellules,intensite)
## Mise en place de l'image issue du diagramme de Voronoï
for i in range(h):
    for j in range(w):
        n = len(cellules)
        cpt = 0
        dist = 1000
        for k in range(n):
            l = len(cellules[k])
            for f in range(l):
                d = distance([i,j],cellules[k][f])
                if(d<dist):
                    dist = d
                    cpt = k
        voronoi[i,j] = intensite[cpt]

## Affichage     
plt.figure()
plt.subplot(121)
plt.imshow(I)
plt.title('image originalle')
plt.subplot(122)               
plt.imshow(voronoi)
plt.title('voronoi')
plt.show() 

############################ Graphe ##########################################################

for k in range(int(maximum)):
    liens.append([])
    pt_control.append([])
    
for i in range(2,h,scale):
    for j in range(2,w,scale):
        liens=Liaison(h,w,i,j,labels_up,scale,I_up,color,liens,pt_control)
        
for k in range(int(maximum)):
    for l in range(len(liens[k])):
        [x0,y0,x1,y1] = [liens[k][l][0][0],liens[k][l][0][1],liens[k][l][1][0],liens[k][l][1][1]]
        plt.plot([y0,y1],[x0,x1], color="red", linewidth=1)   
   
plt.imshow(voronoi)
plt.title('grille')
plt.show()

######################################## B-spline ###################################
       
contour = len(pt_control)
contenant = [] 
for k in range(contour):
    if(len(pt_control[k])>4):
        n_pt_control = get_pt_control(pt_control[k],scale)
        crv1 = BSpline.Curve()
        
        # Set degree
        crv1.degree = 4
        
        # Set control points
        crv1.ctrlpts = n_pt_control
        
        # Generate a uniform knot vector
        crv1.knotvector = knotvector.generate(crv1.degree, crv1.ctrlpts_size)
        contenant.append(crv1)
        

mcrv = multi.CurveContainer(contenant)
# Set the visualization component of the curve container
mcrv.vis = VisMPL.VisCurve2D()

# Plot the curves in the curve container
mcrv.render()


